In [1]:
import sys
# sys.path.append("./subroutines")
sys.path.append("./Vertex/subroutines")

from GAMcalc import *  # Code for calculating vertex functions
import propagator  # propagator object used to store pre-calculated values
import wignerD as wd # wigner D object used to store pre-calculated values

pset=propagator.prop_set(nlam=10) 
wigset = wd.wigner_d_vals()

In [2]:
# import PABS_vertex_calc as PABS
import AB_vertex_calc as bd

In [3]:
# ID = 10001.04006

# looping = "full mark dens comp alt f_bind,"

# settings = np.load(r"C:\Users\tmhab\wlcsim_membrane\binder_diblock\ID=%s_settings.npy" % (ID), allow_pickle = True)

# [chrom, [klog_min, klog_max, klog_num]] = settings

# [n_bind, v_int, Vol_int, e_m, rho_c, rho_s, poly_marks, M, mu_max, mu_min, del_mu, alpha, N, N_m, b] = chrom

# [pa_vec, marks_2] = poly_marks

# mu1_array = np.arange(mu_min, mu_max, del_mu)
# mu2_array = np.arange(mu_min, mu_max, del_mu)

# k_vec = np.logspace(klog_min-1, klog_max, klog_num) / b
# nk = klog_num

# # s2_mats = np.load(r"C:\Users\tmhab\wlcsim_membrane\binder_diblock\ID=%s_chrom_s2_mats_v_int=[[%s,%s],[%s,%s]],diblock,mu_max=%s.npy"\
# #                  % (ID, v_int[0,0], v_int[0,1], v_int[1,0], v_int[1,1], mu_max), allow_pickle = True)

# s_bind_A_arr = np.load(r"ID=%s_s_bind_A.npy" % ID, allow_pickle=True)
# s_bind_B_arr = np.load(r"ID=%s_s_bind_B.npy" % ID, allow_pickle=True)

# spacing = int(1 / del_mu)

In [4]:
ID = 23457.04

# looping = "full mark dens comp alt f_bind,"

settings = np.load(r"C:\Users\tmhab\wlcsim_membrane\binder_diblock\ID=%s_settings.npy" % (ID), allow_pickle = True)

[chrom, [klog_min, klog_max, klog_num]] = settings

# [n_bind, v_int, Vol_int, e_m, rho_c, rho_s, poly_marks, M, mu_max, mu_min, del_mu, f_om, N, N_m, b] = chrom
[n_bind, v_int, Vol_int, e_m, rho_p, rho_s, poly_marks, M, mu_max, mu_min, del_mu, alpha, N, N_m, b] = chrom

[pa_vec, marks_2] = poly_marks

mu1_array = np.arange(mu_min, mu_max, del_mu)
mu2_array = np.arange(mu_min, mu_max, del_mu)

k_vec = np.logspace(klog_min, klog_max, klog_num) / b
  
# s2_mats = np.load(r"C:\Users\tmhab\wlcsim_membrane\binder_diblock\ID=%s_chrom_s2_mats_v_int=[[%s,%s],[%s,%s]],diblock,mu_max=%s.npy"\
#                  % (ID, v_int[0,0], v_int[0,1], v_int[1,0], v_int[1,1], mu_max), allow_pickle = True)

s_bind_A_arr = np.load(r"ID=%s_s_bind_A.npy" % ID, allow_pickle=True)
s_bind_B_arr = np.load(r"ID=%s_s_bind_B.npy" % ID, allow_pickle=True)
spacing = int(1 / del_mu)


print("rho_p (# / nm^3): ", rho_p)
print("rho_s (# / nm^3): ", alpha * rho_p/ M)
print("M: ", M)
print("N: ", N)
print("b: ", b)

print("mu1 array:", mu1_array)
print("v_int: ", v_int)

print("k_vec array:", k_vec)

rho_p (# / nm^3):  3.0100000000000006e-09
rho_s (# / nm^3):  63.99999980736
M:  50
N:  50000
b:  1
mu1 array: [-5.  -4.5 -4.  -3.5 -3.  -2.5 -2.  -1.5 -1.  -0.5]
v_int:  [[-4  4]
 [ 4 -4]]
k_vec array: [0.00316228 0.00356225 0.00401281 0.00452035 0.0050921  0.00573615
 0.00646167 0.00727895 0.00819961 0.00923671 0.01040498 0.01172102
 0.01320352 0.01487352 0.01675475 0.01887392 0.02126112 0.02395027
 0.02697954 0.03039195 0.03423598 0.0385662  0.04344412 0.04893901
 0.0551289  0.06210169 0.06995642 0.07880463 0.08877197 0.1       ]


In [5]:
q_star = 0.3
lam_q = q_star*np.array([1, 0, 0])
Ks = np.array([lam_q, -lam_q, lam_q, -lam_q])
bind_A = np.zeros(len(poly_marks[0]))
bind_A[0:len(bind_A)//2] = 1
bind_B = 1-bind_A

s_bnd_A = bind_A
s_bnd_B = bind_B

FA = 0.5

In [6]:
# np.arange(4*4*4).reshape((4,4,4))

In [7]:
# s3_old = np.arange(4*4*4).reshape((4,4,4))
# s3_old[1:3,1:3,1:3]

# gamma4

In [8]:
G4_quinn = gamma4(pset, wigset, N, FA, Ks)

C:\Users\tmhab\wlcsim_membrane\binder_diblock\./Vertex/subroutines\MultiPoint.py:206: RuntimeWarning: invalid value encountered in scalar divide
  if abs(out/(sum2+tol))<tol and l2>max(lam0_2,lam_2):
C:\Users\tmhab\wlcsim_membrane\binder_diblock\./Vertex/subroutines\MultiPoint.py:209: RuntimeWarning: invalid value encountered in scalar divide
  if abs(sum2/(sum1+tol))<tol and l1>max(lam0_1,lam_1):


Sfs:
S4: [[[[2.20898930e+13+5.38346924e-03j 2.94379138e+10-1.72852955e-01j]
   [2.94379138e+10-1.72852955e-01j 1.10964938e+13+2.71063005e-03j]]

  [[2.94379138e+10-1.72852955e-01j 4.74091447e+06+2.32237829e-09j]
   [1.10964938e+13+2.71063005e-03j 2.94379138e+10-1.72852955e-01j]]]


 [[[2.94379138e+10-1.72852955e-01j 1.10964938e+13+2.71063005e-03j]
   [4.74091447e+06+2.32237829e-09j 2.94379138e+10-1.72852955e-01j]]

  [[1.10964938e+13+2.71063005e-03j 2.94379138e+10-1.72852955e-01j]
   [2.94379138e+10-1.72852955e-01j 2.20898930e+13+5.38346924e-03j]]]]
s3s:
s31: [[[8.32785003e+10+1.01715555e-05j 8.32785003e+10+1.01715555e-05j]
  [1.10723079e+08+2.71193328e-08j 1.10723079e+08+2.71193328e-08j]]

 [[1.10723079e+08+2.71193328e-08j 1.10723079e+08+2.71193328e-08j]
  [8.32785003e+10+1.01715555e-05j 8.32785003e+10+1.01715555e-05j]]]
s32: [[[3.33780260e+08+6.80733426e-08j 3.60368566e+04+3.45797113e-06j]
  [3.71373333e+05-1.72884251e-06j 3.71373333e+05-1.72884251e-06j]]

 [[3.71373333e+05-1.7288425

In [9]:
G4_quinn

(0.4796888144292784-1.400260454643772e-12j)

In [10]:
G4_thomas = bd.gamma4(chrom, s_bnd_A, s_bnd_B, np.array([lam_q, -lam_q, lam_q, -lam_q]))

C:\Users\tmhab\wlcsim_membrane\binder_diblock\PABS_corr_calc.py:495: RuntimeWarning: overflow encountered in exp
  constant = np.exp(-xm_A*(jmax - jupp)- xm_B*(jupp - jlow) - xm_C*(jlow - jmin))
C:\Users\tmhab\wlcsim_membrane\binder_diblock\PABS_corr_calc.py:405: RuntimeWarning: overflow encountered in exp
  constant = np.exp(-xm_A*(jmax - jmid)) * np.exp(-xm_B*(jmid - jmin))
C:\Users\tmhab\wlcsim_membrane\binder_diblock\PABS_corr_calc.py:405: RuntimeWarning: overflow encountered in multiply
  constant = np.exp(-xm_A*(jmax - jmid)) * np.exp(-xm_B*(jmid - jmin))
C:\Users\tmhab\wlcsim_membrane\binder_diblock\PABS_corr_calc.py:405: RuntimeWarning: invalid value encountered in multiply
  constant = np.exp(-xm_A*(jmax - jmid)) * np.exp(-xm_B*(jmid - jmin))


Sfs:
S4: [[[[2.20008593e+13 2.94814815e+10]
   [2.94814815e+10 1.10519309e+13]]

  [[2.94814815e+10 4.93827160e+06]
   [1.10519309e+13 2.94814815e+10]]]


 [[[2.94814815e+10 1.10519309e+13]
   [4.93827160e+06 2.94814815e+10]]

  [[1.10519309e+13 2.94814815e+10]
   [2.94814815e+10 2.20008593e+13]]]]
s3s:
s31: [[[8.31111111e+10 8.31111111e+10]
  [1.11111111e+08 1.11111111e+08]]

 [[1.11111111e+08 1.11111111e+08]
  [8.31111111e+10 8.31111111e+10]]]
s32: [[[3.30888889e+08 3.70370370e+04]
  [3.70370370e+05 3.70370370e+05]]

 [[3.70370370e+05 3.70370370e+05]
  [1.51062874e-02 3.30888889e+08]]]
s33: [[[8.31111111e+10 8.31111111e+10]
  [1.11111111e+08 1.11111111e+08]]

 [[1.11111111e+08 1.11111111e+08]
  [8.31111111e+10 8.31111111e+10]]]
s2s:
s2inv: [[ 3.00802677e-07 -4.02142616e-10]
 [-4.02142616e-10  3.00802677e-07]]
s21inv: [[4.e-10+0.j 4.e-10+0.j]
 [4.e-10+0.j 4.e-10+0.j]]
s22inv: [[ 1.20080067e-06 -4.00533912e-10]
 [-4.00533912e-10  1.20080067e-06]]
s23inv: [[4.e-10+0.j 4.e-10+0.j]
 [4.e-

In [11]:
N

50000

In [12]:
-G4_thomas*N**3

(0.48079945164301446+0j)

In [13]:
G4_quinn

(0.4796888144292784-1.400260454643772e-12j)

# gamma 2

In [14]:
CHI = 10

In [15]:
G2_quinn = gamma2(pset, N, FA, [0.3], CHI)

In [16]:
G2_thomas = bd.gamma2(chrom, s_bnd_A, s_bnd_B, lam_q, CHI)

chi divided by N


In [17]:
G2_thomas*N

array([[-19.96987952]])

In [18]:
G2_quinn

(-19.969940281155488-3.666551045619528e-18j)

# lam free E

In [20]:
amp_l_quinn = optimize.fmin(lambda amps: np.real(amps**2 * G2_quinn + amps**4 * (1/24) * (6) * (1) * G4_quinn), \
                      1, disp=False)

lamF_quinn = amp_l_quinn**2 * G2_quinn + amp_l_quinn**4 *  (1/24) * (6) * (1) *G4_quinn 
lamF_quinn

array([-831.36921862-2.42685147e-09j])

In [48]:
amp_l_quinn

array([9.12480469])

In [21]:
amp_l_thomas = optimize.fmin(lambda amps: np.real(amps**2 * G2_thomas - amps**4 * (1/24) * (6) * (1) * G4_thomas), \
                      1, disp=False)

lamF_thomas = amp_l_thomas**2 * G2_thomas - amp_l_thomas**4 *  (1/24) * (6) * (1) *G4_thomas 
lamF_thomas

array([[-41472186.23102992+0.j]])

In [22]:
amp_l_thomas

array([455712.2921875])

In [23]:
# CONCLUSION:
# it is key to have chi/N term in gamma2 in order to get finite negative free energy
# also key to make my gamma 4 negative